In [1]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
from scipy import stats
import pickle 
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union,Dict,Set
from hail.plot import show, output_notebook
from bokeh.palettes import d3  # pylint: disable=no-name-in-module
from bokeh.models import Plot, Row, Span, NumeralTickFormatter, LabelSet
from gnomad.utils.plotting import *
from typing import Set, Tuple

tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
plot_dir = "/home/ubuntu/data/tmp"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.5
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.41-b8144dba46e6
LOGGING: writing to /home/ubuntu/data/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20201214-1510-0.2.41-b8144dba46e6.log


Loading BokehJS ...

## Filtered mt with AC <=2 and synonymous variants only

In [2]:
#mt_filtered = mt_trios.filter_rows((mt_trios.info.AC[0] <= 2) & (
    #    mt_trios.consequence == "synonymous_variant"))
#Filtered mt with the above filter AC <=2 and synonymous variants only
mt_filtered = hl.read_matrix_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/sanger_cohorts_AC_synonymous_filtered.mt')

In [3]:
mt_trans=mt_filtered.filter_entries(mt_filtered.info.AC[0]==2)
mt_untrans=mt_filtered.filter_entries(mt_filtered.info.AC[0]==1)


In [4]:
mt_trans_count=mt_trans.group_cols_by(mt_trans.id).aggregate(transmitted_singletons_count=hl.agg.count_where((mt_trans.info.AC[0] == 2) &
                                (mt_trans.proband_entry.GT.is_het_ref()) &
                                (mt_trans.father_entry.GT.is_het_ref()) |
                                (mt_trans.mother_entry.GT.is_het_ref())))

In [5]:
Total_transmitted_singletons=mt_trans_count.aggregate_entries(hl.agg.count_where(mt_trans_count.transmitted_singletons_count==1))
print(Total_transmitted_singletons)

28918


In [19]:
mt_untrans_count = (mt_untrans.group_cols_by(mt_untrans.id).aggregate(
    untransmitted_singletons_count=hl.agg.count_where((mt_untrans.info.AC[0] == 1) &
                     (mt_untrans.proband_entry.GT.is_hom_ref()) &
                     (mt_untrans.father_entry.GT.is_het_ref()) |
                     (mt_untrans.mother_entry.GT.is_het_ref()))))

In [10]:
Total_untransmitted_singletons=mt_untrans_count.aggregate_entries(hl.agg.count_where(mt_untrans_count.untransmitted_singletons_count==1))
print(Total_untransmitted_singletons)

24816


In [12]:
Ratio_transmitted_untransmitted=Total_transmitted_singletons/Total_untransmitted_singletons
print(Ratio_transmitted_untransmitted)

1.1652965828497743


In [6]:
mt_trans_count.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    'id': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: flo

In [20]:
mt2=mt_trans_count.annotate_rows(variant_transmitted_singletons=hl.agg.count_where(mt_trans_count.transmitted_singletons_count==1))

In [21]:
mt2

In [22]:
mt2.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    'id': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: flo

In [18]:
mt2.variant_transmitted_singletons.summarize()

Non-missing,225722 (100.00%)
Missing,0
Minimum,0
Maximum,4
Mean,0.13
Std Dev,0.35


In [23]:
mt3=mt_untrans_count.annotate_rows(variant_untransmitted_singletons=hl.agg.count_where(mt_untrans_count.untransmitted_singletons_count==1))

In [24]:
mt3.variant_untransmitted_singletons.summarize()

Non-missing,225722 (100.00%)
Missing,0
Minimum,0
Maximum,4
Mean,0.11
Std Dev,0.32


In [25]:
run_hash="91b132aa"
ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/{run_hash}/{run_hash}_rf_result_sanger_cohorts_DENOVO_family_stats_SYNONYMOUS.ht')


In [29]:
ht=ht.annotate(variant_transmitted_singletons=mt2.rows()[ht.key].variant_transmitted_singletons)

In [30]:
ht=ht.annotate(variant_untransmitted_singletons=mt3.rows()[ht.key].variant_untransmitted_singletons)

In [31]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        n_alt_alleles: int32, 
        QD: float64, 
        MQRankSum: float64, 
        SOR: float64, 
        ReadPosRankSum: float64, 
        FS: float64, 
        DP: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float32 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'QD': float64 
    'MQRankSum': float64 
    'SOR': float64 
    'ReadPosRankSum': 

In [32]:
ht.write(f'{tmp_dir}/ddd-elgh-ukbb/variant_qc/models/{run_hash}/{run_hash}_rf_result_transmitted_singletons.ht', overwrite=True)

2020-12-14 15:42:42 Hail: INFO: wrote table with 13669739 rows in 500 partitions to hdfs://spark-master:9820//ddd-elgh-ukbb/variant_qc/models/91b132aa/91b132aa_rf_result_transmitted_singletons.ht


In [33]:
ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/{run_hash}/{run_hash}_rf_result_transmitted_singletons.ht')


In [34]:
ht.summarize()

Non-missing,13669739 (100.00%)
Missing,0
Contig Counts,"{'chr5': 1456829, 'chr8': 1384, 'chr19': 1485, 'chr1': 3001881, 'chr20': 785942, 'chr2': 2275391, 'chr7': 1564981, 'chr3': 1797470, 'chr4': 1307270, 'chr6': 1477106}"
Non-missing,13669739 (100.00%)
Missing,0
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,27339478 (100.00%)
Missing,0
Min Size,1
